# Memoria Práctica 3

Autores: Román García y Patricia Losana

In [ ]:
from Datos import Datos
from EstrategiaParticionado import *
from Clasificador import *
from ClasificadorAG import *
from Roc import *
import numpy as np
from sklearn import preprocessing 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import pprint
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import accuracy_score
from plotModel import plotModel
import matplotlib.pyplot as plt

## 1. Implementación

A continuación se muestran algunos detalles de la implementación llevada a cabo del algoritmo genético.

### 1.1 Generación de la población inicial

Para generar la población inicial es necesario especificar el set de datos a utilizar, y el número de individuos y de generaciones que se desea. 

~~~~
ag = ClasificadorAG(n_cromosomas, dataset, n_generaciones)
~~~~

En ese momento se creará una instancia de la clase ClasificadorAG, donde se generará la población inicial de individuos (cromosomas) con el método generarPoblacion(). 

~~~~
def generarPoblacion(self, n_cromosomas, dataset, regla_entera):
    for _ in range(n_cromosomas):
        self.cromosomas.append(self.Cromosoma(dataset = dataset, regla_entera=regla_entera))
~~~~

Ese método guarda un array de objetos de la clase Cromosoma (donde cada uno de los cuales es un individuo) en el clasificador. Se ha establecido cromosoma como un conjunto de reglas distintas (set()): 
~~~~
class Cromosoma:
    def __init__(self, dataset, reglas = None, regla_entera = True):
        self.regla_entera = regla_entera
        self.n_attrs = len(dataset.nombreAtributos) - 1  #nº attrs menos la clase
        self.rlen = randint(1, pow(dataset.k, self.n_attrs))
        self.datos = dataset.convertirAIntervalos(dataset.datos)
        self.n_intervalos = dataset.k
        self.dataset = dataset
        if reglas != None:
            self.reglas = reglas
        else:
            self.reglas = set()
            self.generarReglas(dataset)

        self.fit = -1
~~~~
El método generarReglas crea objetos de la clase Regla dentro del cromosoma. 

**Número de reglas:** Para cada cromosoma se ha establecido un número aleatorio entre 1 y 100. Se ha probado también con un número aleatorio entre 1 y 20, y también con todas las posibles combinaciones distintas de intervalos para cada atributo (nº atributos^(k + 1)). Aunque el último es el que debería tener una mejor predicción, no ha sido viable a nivel computacional. Entre 1 y 100 sí ha sido viable y los resultados son significativamente mejores que con cromosomas que puedan tener hasta 20 reglas. El principal inconveniente es su elevado coste a nivel temporal.

~~~~
class Regla():
    def __init__(self,dataset, regla_entera = True):
        self.regla_entera = regla_entera
        self.valores = []
        self.n_intervalos = dataset.k  #nº attrs menos la clase

        if regla_entera:
            self.valores = np.append(np.random.randint(dataset.k + 1, size = len(dataset.nombreAtributos)-1), np.random.randint(len(dataset.diccionarios['Class']), size = 1))
        else:
            regla = []
            for _ in range(len(dataset.nombreAtributos)-1):
                regla.append(np.random.randint(2, size = dataset.k).tolist())
            regla.append(np.random.randint(len(dataset.diccionarios['Class']), size = 1)[0])
            self.valores = regla
~~~~          
                    

### 1.2 Mecanismo de cruce implementado

El mecanismo de cruce implementado es entre dos cromosomas por un punto de cruce. 

El proceso es el siguiente: para cada cromosoma se llama al método recombinar cromosomas, el cual lo recombina otro cromosoma seleccionado mediante el mecanismo de la ruleta.
~~~~
def recombinar(self):
    poblacion = []
    ruleta = ClasificadorAG.ruleta_rusa(self.cromosomas,len(self.cromosomas))
    for cromosoma in self.cromosomas:
        poblacion.append(cromosoma.recombinar(ruleta.pop(random.randint(0, len (ruleta) - 1))))
    return poblacion


def ruleta_rusa(cromosomas, n_individuos):
    seleccion, probabilidades = [], []

    total_fitness = 0
    for cromosoma in cromosomas:
        total_fitness += cromosoma.fitness()

    if total_fitness == 0:
        return cromosomas
    for cromosoma in cromosomas:
        probabilidades.append(cromosoma.fitness()/total_fitness)
    seleccion = np.random.choice(cromosomas, n_individuos, p=probabilidades)
    return seleccion.tolist()
~~~~
Posteriormente, dentro de la clase cromosoma, se recombinará con una probabilidad de cruce del 80%: es decir, si al extraer un número aleatorio de 0 a 99 sale un número inferior a 80, se procederá a recombinar a los dos cromosomas.


~~~~   
def recombinar(self, cromosoma):
    if tirarDado(99) < 80:
        return self.recombinar_cromosomas(cromosoma)
    return self

def tirarDado(caras, ini = 0):
    return random.randint(ini,caras)
~~~~

Finalmente, si se produce la recombinación, se intercambia la primera mitad del cromosoma 1 con la segunda mitad del cromosoma 2 y viceversa. El resultado de esta combinación se devuelve al clasificador.
~~~~
def recombinar_cromosomas(self, cromosoma):
    medio = round(len(self.reglas)/2)
    medio_other = round(len(cromosoma.reglas)/2)

    return ClasificadorAG.Cromosoma(self.dataset, reglas = set(list(self.reglas)[:medio] + list(cromosoma.reglas)[:medio_other]), regla_entera = self.regla_entera )
~~~~


### 1.3 Mecanismo de mutación implementado

El mecanismo de mutación implementado es un bitflip, y la estrategia es parecida a la de recombinación: para cada cromosoma del clasificador se llama al método mutar del cromosoma. 
~~~~
def mutar(self):
    poblacion = []
    for cromosoma in self.cromosomas:
        cromosoma.mutar()
~~~~
Ese cromosoma tendrá un 4% de probabilidades de mutar cualquier regla, es decir: para cada regla del cromosoma, si al extraer un número aleatorio de 0 a 99 sale un número inferior a 4 (valor por defecto), se procederá a mutar esa regla.
~~~~
def mutar(self, porcentaje = 4):
    for regla in self.reglas:
        if tirarDado(99) < porcentaje:
            regla.mutar(porcentaje)
~~~~
Finalmente, dentro de la regla concreta a mutar, cada posición de la regla tendrá de nuevo un 4% de probabilidades de mutar, es decir: si al extraer un número aleatorio de 0 a 99 saliese un número inferior a 4 (valor por defecto), se asigna a esa posición de la regla otro número al azar entre 0 y el número máximo del intervalo.
~~~~
def mutar(self, porcentaje = 4):
    if self.regla_entera:
        for indice, intervalo in enumerate(self.valores[:-1]):
            if tirarDado(99) > porcentaje:
                self.valores[indice] = tirarDado(self.n_intervalos)
    else: 
        for index_attr, attr in enumerate(self.valores[:-1]):
            for index_intervalo, intervalo in enumerate(attr):
                if tirarDado(99) > porcentaje:
                    self.valores[index_attr][index_intervalo] = tirarDado(1)
~~~~

## 2. Resultados de la clasificación

A continuación se muestran los resultados de la clasificación del algoritmo genético para los datasets example1.data, example3.data, example4.data y wdbc-10.data (una versión reducida sin los últimos 10 atributos de wdbc).

Para cada dataset se muestran los siguientes resultados, tanto con reglas enteras como con reglas binarias: 

 * Tamaño de población = 100 ; Generaciones = 100
 * Tamaño de población = 100 ; Generaciones = 500
 * Tamaño de población = 100 ; Generaciones = 1000
 * Tamaño de población = 200 ; Generaciones = 100
 * Tamaño de población = 200 ; Generaciones = 500
 * Tamaño de población = 200 ; Generaciones = 1000 


In [ ]:
generaciones = [100,500, 1000]
poblaciones = [10,200]

### Resultados para example1.data

In [ ]:
dataset = Datos("ConjuntosDatos/example1.data")

a, k = dataset.crearIntervalos(dataset.datos)
for poblacion in poblaciones:
    for generacion in generaciones:
        print("-------------------------------------------------------------------")
        print("Reglas enteras: \n")
        print("Tamaño de la poblacion = ", poblacion, "\nGeneración = ", generacion)

        ag = ClasificadorAG(poblacion, dataset, generacion)
        val = ag.validacion(ValidacionSimple(), dataset, ag)
        print(val)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))
        
        plt.plot(range(generacion), f_hulk)
        plt.title('Evolución del fitness del mejor individuo')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        plt.plot(range(generacion), f_medio)
        plt.title('Evolución del fitness medio de la población')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        print("\nReglas binarias: \n")
        print("Tamaño de la poblacion = ", poblacion, "\nGeneración = ", generacion)
        ag = ClasificadorAG(poblacion, dataset, generacion, regla_entera = False)
        val = ag.validacion(ValidacionSimple(), dataset, ag)
        print("\nErrores =", val)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))

        plt.plot(range(generacion), f_hulk)
        plt.title('Evolución del fitness del mejor individuo')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        plt.plot(range(generacion), f_medio)
        plt.title('Evolución del fitness medio de la población')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()

### Resultados para example3.data

In [ ]:
dataset = Datos("ConjuntosDatos/example3.data")

a, k = dataset.crearIntervalos(dataset.datos)
for poblacion in poblaciones:
    for generacion in generaciones:
        print("-------------------------------------------------------------------")
        print("Reglas enteras: \n")
        print("Tamaño de la poblacion = ", poblacion, "; \nGeneración = ", generacion)

        ag = ClasificadorAG(poblacion, dataset, generacion)
        val = ag.validacion(ValidacionSimple(), dataset, ag)
        print(val)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))
        
        plt.plot(range(generacion), f_hulk)
        plt.title('Evolución del fitness del mejor individuo')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        plt.plot(range(generacion), f_medio)
        plt.title('Evolución del fitness medio de la población')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        print("\nReglas binarias: \n")
        print("Tamaño de la poblacion = ", poblacion, "; \nGeneración = ", generacion)
        ag = ClasificadorAG(poblacion, dataset, generacion, regla_entera = False)
        val = ag.validacion(ValidacionSimple(), dataset, ag)
        print("\nErrores =", val)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))

        plt.plot(range(generacion), f_hulk)
        plt.title('Evolución del fitness del mejor individuo')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        plt.plot(range(generacion), f_medio)
        plt.title('Evolución del fitness medio de la población')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()

### Resultados para example4.data

In [ ]:
dataset = Datos("ConjuntosDatos/example4.data")

a, k = dataset.crearIntervalos(dataset.datos)
for poblacion in poblaciones:
    for generacion in generaciones:
        print("-------------------------------------------------------------------")
        print("Reglas enteras: \n")
        print("Tamaño de la poblacion = ", poblacion, "; \nGeneración = ", generacion)

        ag = ClasificadorAG(poblacion, dataset, generacion)
        val = ag.validacion(ValidacionSimple(), dataset, ag)
        print(val)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))
        
        plt.plot(range(generacion), f_hulk)
        plt.title('Evolución del fitness del mejor individuo')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        plt.plot(range(generacion), f_medio)
        plt.title('Evolución del fitness medio de la población')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        print("\nReglas binarias: \n")
        print("Tamaño de la poblacion = ", poblacion, "; \nGeneración = ", generacion)
        ag = ClasificadorAG(poblacion, dataset, generacion, regla_entera = False)
        val = ag.validacion(ValidacionSimple(), dataset, ag)
        print("\nErrores =", val)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))
        
        plt.plot(range(generacion), f_hulk)
        plt.title('Evolución del fitness del mejor individuo')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        plt.plot(range(generacion), f_medio)
        plt.title('Evolución del fitness medio de la población')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()

### Resultados para wdbc-10.data

In [ ]:
dataset = Datos("ConjuntosDatos/wdbc-10.data")

a, k = dataset.crearIntervalos(dataset.datos)
for poblacion in poblaciones:
    for generacion in generaciones:
        print("-------------------------------------------------------------------")
        print("Reglas enteras: \n")
        print("Tamaño de la poblacion = ", poblacion, "; \nGeneración = ", generacion)

        ag = ClasificadorAG(poblacion, dataset, generacion)
        val = ag.validacion(ValidacionSimple(), dataset, ag)
        print(val)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))
        
        plt.plot(range(generacion), f_hulk)
        plt.title('Evolución del fitness del mejor individuo')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        plt.plot(range(generacion), f_medio)
        plt.title('Evolución del fitness medio de la población')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        print("\nReglas binarias: \n")
        print("Tamaño de la poblacion = ", poblacion, "; \nGeneración = ", generacion)
        ag = ClasificadorAG(poblacion, dataset, generacion, regla_entera = False)
        val = ag.validacion(ValidacionSimple(), dataset, ag)
        print("\nErrores =", val)
        print("\nPromedio del error = ", round(np.array(val).mean(),4), "\tDesviación típica = ", round(np.array(val).std(), 4))

        plt.plot(range(generacion), f_hulk)
        plt.title('Evolución del fitness del mejor individuo')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()
        
        plt.plot(range(generacion), f_medio)
        plt.title('Evolución del fitness medio de la población')
        plt.xlabel('numero de generaciones')
        plt.ylabel('fitness')
        plt.grid()
        plt.show()

## 3. Análisis de los resultados

Importancia del número de reglas

Importancia del tamaño de la población

Importancia de las generaciones

Importancia de las tasas de cruce y mutación

Importancia de la representación (enteros o cadenas binarias)

## 4. Representaciones gráficas

### a) Evolución del fitness del mejor individuo

Para cada generación mostrar en pantalla el número de generación y el fitness del mejor individuo

### b) Evolución del fitness medio de la población

Para cada generación mostrar en pantalla el número de generación y el fitness del mejor individuo